## 6.01 - Supervised Learning Model Comparison

Recall the "data science process."

1. Define the problem.
2. Gather the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

In this lab, we're going to focus mostly on creating (and then comparing) many regression and classification models. Thus, we'll define the problem and gather the data for you.
Most of the questions requiring a written response can be written in 2-3 sentences.

### Step 1: Define the problem.

You are a data scientist with a financial services company. Specifically, you want to leverage data in order to identify potential customers.

If you are unfamiliar with "401(k)s" or "IRAs," these are two types of retirement accounts. Very broadly speaking:
- You can put money for retirement into both of these accounts.
- The money in these accounts gets invested and hopefully has a lot more money in it when you retire.
- These are a little different from regular bank accounts in that there are certain tax benefits to these accounts. Also, employers frequently match money that you put into a 401k.
- If you want to learn more about them, check out [this site](https://www.nerdwallet.com/article/ira-vs-401k-retirement-accounts).

We will tackle one regression problem and one classification problem today.
- Regression: What features best predict one's income?
- Classification: Predict whether or not one is eligible for a 401k.

Check out the data dictionary [here](http://fmwww.bc.edu/ec-p/data/wooldridge2k/401KSUBS.DES).

### NOTE: When predicting `inc`, you should pretend as though you do not have access to the `e401k`, the `p401k` variable, and the `pira` variable. When predicting `e401k`, you may use the entire dataframe if you wish.

### Step 2: Gather the data.

##### 1. Read in the data from the repository.

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor



In [2]:
df_401k = pd.read_csv('./401ksubs.csv')

In [3]:
df_401k.isnull().sum()

e401k     0
inc       0
marr      0
male      0
age       0
fsize     0
nettfa    0
p401k     0
pira      0
incsq     0
agesq     0
dtype: int64

In [4]:
df_401k.head()

,e401k,inc,marr,male,age,fsize,nettfa,p401k,pira,incsq,agesq
0,0,13.170,0,0,40,1,4.575,0,1,173.4489,1600
1,1,61.230,0,1,35,1,154.000,1,0,3749.1130,1225
2,0,12.858,1,0,44,2,0.000,0,0,165.3282,1936
3,0,98.880,1,1,44,2,21.800,0,0,9777.2540,1936
4,0,22.614,0,0,53,1,18.450,0,0,511.3930,2809


##### 2. What are 2-3 other variables that, if available, would be helpful to have?

Occupation, education level

##### 3. Suppose a peer recommended putting `race` into your model in order to better predict who to target when advertising IRAs and 401(k)s. Why would this be an unethical decision?

The model may predict advertising to one race over another, possibly creating a barrier to access, or creating a bias within the team/company on who to target for financial services. We would not want to avoid targeting a certain group of people due to their race. 

## Step 3: Explore the data.

##### 4. When attempting to predict income, which feature(s) would we reasonably not use? Why?

We would want to make sure that some of the variables are not collinear. For example, marriage and family size may be too highly correlated to include both in the model. We would want to test them for collinearity before including them. Similary, Age^2 seems to be an engineered feature that would be directly correlated to Age, so we would not want to include age, just Age^2.

##### 5. What two variables have already been created for us through feature engineering? Come up with a hypothesis as to why subject-matter experts may have done this.
> This need not be a "statistical hypothesis." Just brainstorm why SMEs might have done this!

https://stats.stackexchange.com/questions/52585/what-happens-when-i-include-a-squared-variable-in-my-regression

agesq, incsq are squared versions of the variables. This may be because income eventually will reach a plateau.

Squaring the data will also standardaize variables like age, since they are so far away from income it would be helpful to make them closer together to understand their relationship. 

##### 6. Looking at the data dictionary, one variable description appears to be an error. What is this error, and what do you think the correct value would be?

both age^2 and inc^2 are in the data dictionary twice. When examining the data, it seems that the variables are correct though. 

## Step 4: Model the data. (Part 1: Regression Problem)

Recall:
- Problem: What features best predict one's income?
- When predicting `inc`, you should pretend as though you do not have access to the `e401k`, the `p401k` variable, and the `pira` variable.

##### 7. List all modeling tactics we've learned that could be used to solve a regression problem (as of Wednesday afternoon of Week 6). For each tactic, identify whether it is or is not appropriate for solving this specific regression problem and explain why or why not.

**Multiple Linear regression** - multiple variables to predict y and interpretable cofficients make this model very useful in this situation. Since we are predicting a numeric variable, we can use MLR.

**Logistic regression** - this predicts categories, so it would not be helpful in this problem. 

**KNearest Neighbors** - non-parametric (not making assumptions about the distribution of the data), uses distance so the data needs to be scaled, is difficult to interpret the features which is importnat to our model because we want to understand the features. 

**Decision Tree Regression** - We don't have to scale the data, it is prone to overfitting, and is easy to interpret the features, which is what we are looking for in this question. 

**Random Forests and ExtraTrees** - This de-correlates the decision tree regression and helps take away variance by employing random subset sampling in the algorithm. If the Decision tree regression is too overfit, it is useful to employe these to help reduce variance and strenghten the model. 

**Adaboost Model** - An ensemble method of modeling that assigns weights to specific features with the goal of correct misclassifications

**Suppor Vector Regressor** - Black box model, which is not helpful for interpretation. 

##### 8. Regardless of your answer to number 7, fit at least one of each of the following models to attempt to solve the regression problem above:
    - a multiple linear regression model
    - a k-nearest neighbors model
    - a decision tree
    - a set of bagged decision trees
    - a random forest
    - an Adaboost model
    - a support vector regressor
    
> As always, be sure to do a train/test split! In order to compare modeling techniques, you should use the same train-test split on each. I recommend setting a random seed here.

> You may find it helpful to set up a pipeline to try each modeling technique, but you are not required to do so!

In [5]:
np.random.seed(42)

In [12]:
X =df_401k[['marr','male','age','fsize','nettfa','incsq','agesq']]
y= df_401k['inc']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

#### Multiple Linear Regression

In [14]:
#instantiate the Model
lr = LinearRegression()

#fit the model 
lr.fit(X_train, y_train)

#score the  data
print(f'Training Score: {lr.score(X_train, y_train)}, Testing Score: {lr.score(X_test, y_test)}')

Training Score: 0.8948254673897011, Testing Score: 0.9055024120733456


#### K-Nearest Neighbor

In [15]:
sc = StandardScaler()

In [16]:
sc = StandardScaler()
Z_train = sc.fit_transform(X_train)
Z_test = sc.transform(X_test)

In [22]:
knnr = KNeighborsRegressor(n_neighbors=5)
knnr.fit(Z_train, y_train)

KNeighborsRegressor()

In [29]:
print(f'Training Score: {knnr.score(Z_train, y_train)},Testing Score: {knnr.score(Z_test, y_test)}')

Training Score: 0.9795704183969427,Testing Score: 0.9728418554699854


### Decision Tree

In [30]:
from sklearn.tree import DecisionTreeRegressor

In [31]:
dtr = DecisionTreeRegressor()

In [32]:
dtr.fit(Z_train, y_train)

DecisionTreeRegressor()

In [33]:
dtr.score(Z_train, y_train), dtr.score(Z_test, y_test)

(1.0, 0.9999481486457086)

### Bagged Decision Tree

In [37]:
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html

from sklearn.ensemble import BaggingRegressor

In [39]:
bgr = BaggingRegressor(random_state = 42, base_estimator = DecisionTreeRegressor())

In [40]:
bgr.fit(Z_train, y_train)

BaggingRegressor(base_estimator=DecisionTreeRegressor(), random_state=42)

In [41]:
bgr.score(Z_train, y_train), bgr.score(Z_test, y_test)

(0.999989416529091, 0.9999883790392515)

### Random Forest

In [45]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

In [46]:
rfr = RandomForestRegressor(n_estimators = 100)

In [47]:
rfr.fit(Z_train, y_train)

RandomForestRegressor()

In [50]:
rfr.score(Z_train, y_train),rfr.score(Z_test, y_test)

(0.9999889348368476, 0.9999907283001671)

### Ada Boosting

In [51]:
from sklearn.ensemble import AdaBoostRegressor

In [52]:
abr = AdaBoostRegressor(random_state = 42)

In [53]:
abr.fit(Z_train, y_train)

AdaBoostRegressor(random_state=42)

In [55]:
abr.score(Z_train, y_train)

0.9913321095472987

In [56]:
abr.score(Z_test, y_test)

0.9916962982297579

In [58]:
abr.estimators_[:5]

[DecisionTreeRegressor(max_depth=3, random_state=1608637542),
 DecisionTreeRegressor(max_depth=3, random_state=1776589882),
 DecisionTreeRegressor(max_depth=3, random_state=1666910835),
 DecisionTreeRegressor(max_depth=3, random_state=754668208),
 DecisionTreeRegressor(max_depth=3, random_state=1177281088)]

### Support Vector Regressor

In [59]:
from sklearn.svm import LinearSVR

In [60]:
svr = LinearSVR(random_state=42, max_iter=5000)

In [61]:
svr.fit(Z_train, y_train)

LinearSVR(max_iter=5000, random_state=42)

In [62]:
svr.score(Z_train, y_train), svr.score(Z_test, y_test)

(0.8149823407006432, 0.8266033728325769)

##### 9. What is bootstrapping?

Bootstrapping is a resampling method by which the model chooses random samples of the population then replaces them each time. The model is fit on each new bootstrapped sample. All of those models are then combined.

##### 10. What is the difference between a decision tree and a set of bagged decision trees? Be specific and precise!

A decision tree is a model that finds rules based on the X dataset and then splits the data into smaller subset, each time making a decision on how to split the data. The model makes the decision based on 

##### 11. What is the difference between a set of bagged decision trees and a random forest? Be specific and precise!

##### 12. Why might a random forest be superior to a set of bagged decision trees?
> Hint: Consider the bias-variance tradeoff.

## Step 5: Evaluate the model. (Part 1: Regression Problem)

##### 13. Using RMSE, evaluate each of the models you fit on both the training and testing data.

##### 14. Based on training RMSE and testing RMSE, is there evidence of overfitting in any of your models? Which ones?

##### 15. Based on everything we've covered so far, if you had to pick just one model as your final model to use to answer the problem in front of you, which one model would you pick? Defend your choice.

##### 16. Suppose you wanted to improve the performance of your final model. Brainstorm 2-3 things that, if you had more time, you would attempt.

## Step 4: Model the data. (Part 2: Classification Problem)

Recall:
- Problem: Predict whether or not one is eligible for a 401k.
- When predicting `e401k`, you may use the entire dataframe if you wish.

##### 17. While you're allowed to use every variable in your dataframe, mention at least one disadvantage of using `p401k` in your model.

##### 18. List all modeling tactics we've learned that could be used to solve a classification problem (as of Wednesday afternoon of Week 6). For each tactic, identify whether it is or is not appropriate for solving this specific classification problem and explain why or why not.

##### 19. Regardless of your answer to number 18, fit at least one of each of the following models to attempt to solve the classification problem above:
    - a logistic regression model
    - a k-nearest neighbors model
    - a decision tree
    - a set of bagged decision trees
    - a random forest
    - an Adaboost model
    - a support vector classifier
    
> As always, be sure to do a train/test split! In order to compare modeling techniques, you should use the same train-test split on each. I recommend using a random seed here.

> You may find it helpful to set up a pipeline to try each modeling technique, but you are not required to do so!

## Step 5: Evaluate the model. (Part 2: Classfication Problem)

##### 20. Suppose our "positive" class is that someone is eligible for a 401(k). What are our false positives? What are our false negatives?

##### 21. In this specific case, would we rather minimize false positives or minimize false negatives? Defend your choice.

##### 22. Suppose we wanted to optimize for the answer you provided in problem 21. Which metric would we optimize in this case?

##### 23. Suppose that instead of optimizing for the metric in problem 21, we wanted to balance our false positives and false negatives using `f1-score`. Why might [f1-score](https://en.wikipedia.org/wiki/F1_score) be an appropriate metric to use here?

##### 24. Using f1-score, evaluate each of the models you fit on both the training and testing data.

##### 25. Based on training f1-score and testing f1-score, is there evidence of overfitting in any of your models? Which ones?

##### 26. Based on everything we've covered so far, if you had to pick just one model as your final model to use to answer the problem in front of you, which one model would you pick? Defend your choice.

##### 27. Suppose you wanted to improve the performance of your final model. Brainstorm 2-3 things that, if you had more time, you would attempt.

## Step 6: Answer the problem.

##### BONUS: Briefly summarize your answers to the regression and classification problems. Be sure to include any limitations or hesitations in your answer.

- Regression: What features best predict one's income?
- Classification: Predict whether or not one is eligible for a 401k.